In [1]:
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

import os
import re

In [2]:
# create glob object for both train and test directories
train_img_glob = glob('data/train/*/*.jpg')
test_img_glob = glob('data/test/*/*.jpg')

# train and test directories
base = os.path.dirname('')
TRAIN_DIR = os.path.join(base + 'data/train')
TEST_DIR = os.path.join(base + 'data/test')

# initiate model and weights for extracting faces
prototxt_file = os.path.join(base + 'data/deploy.prototxt.txt')
caffemodel_file = os.path.join(base + 'data/weights.caffemodel')

In [3]:
# count total images from both directories
total = len(train_img_glob) + len(test_img_glob)
percent_train = int(len(train_img_glob)/total * 100)
percent_test = int(len(test_img_glob)/total * 100)

print("Total images:", total)
print("Total number of training images before preprocessing:", len(train_img_glob), "(", percent_train, "%)")
print("Total number of training images before preprocessing:", len(test_img_glob), "(", percent_test, "%)")

Total images: 1810
Total number of training images before preprocessing: 1480 ( 81 %)
Total number of training images before preprocessing: 330 ( 18 %)


In [4]:
# use cv2 built-in method cv2.dnn.readNetFromCaffe to extract faces
face_model = cv2.dnn.readNetFromCaffe(prototxt_file, caffemodel_file)

In [5]:
print('Initiating face extraction process (Training)...')

SAVE = './data/faces/train/'
CONFIDENCE = 0.5
train_skipped = 0

count = 0
last = 'dahyun'
# training image
print('[STATUS] Reading & extracting image...')
for img in tqdm(train_img_glob):
    
    label = re.findall('\\\\[a-z].*\\\\', img)[0]
    label = label[1:-1]
    
    if not os.path.isdir(os.path.join(SAVE, label)):
        os.mkdir(os.path.join(SAVE, label))
    
    if label != last:
        count = 0
        last = label
    else:
        count = count + 1
    
    # read in the image
    a = cv2.imread(img)
    (h,w) = a.shape[:2]
    # create a blob object
    blob = cv2.dnn.blobFromImage(a, scalefactor=1.0, size=(224, 224), 
                                 mean=(104.0, 177.0, 123.0), swapRB=False, crop=False)
    face_model.setInput(blob)
    # detect face
    detector = face_model.forward() 
    
    # if any face is detected
    if len(detector) > 0:
        # get the index of the face
        i = np.argmax(detector[0,0,:,2])
        confidence = detector[0,0,i,2]
        
        # if the confidence is higher than the threshold we set earlier
        if confidence > CONFIDENCE:
            # extract face from a rectangle
            rect = detector[0,0,i,3:7] * np.array([w,h,w,h])
            start_x, start_y, end_x, end_y = rect.astype('int')
            
            face = a[start_y:end_y, start_x:end_x]
            
            # skip it if there is no face
            if face.size == 0:
                print('[INFO] Skipping... No face detected:', img)
                train_skipped += 1
                continue
            else:
                img_file_name = label + '_face_' + str(count) + '.jpg'
#                 print(SAVE + label + '/' + img_file_name)
#                 print(os.path.join(SAVE, label, img_file_name))
                cv2.imwrite(SAVE + label + '/' + img_file_name, face)
        
        # skip otherwise
        else:
            print('[INFO] Skipping... Confidence below threshold:', img)
            train_skipped += 1
            continue

print('[STATUS] Face extraction process completed!')

  0%|                                                                                         | 0/1480 [00:00<?, ?it/s]

Initiating face extraction process (Training)...
[STATUS] Reading & extracting image...


  2%|█▊                                                                              | 33/1480 [00:00<00:55, 25.93it/s]

[INFO] Skipping... No face detected: data/train\dahyun\RCPC_members-1209682000044445697-20191225_104718-img3.jpg


  4%|███▏                                                                            | 60/1480 [00:02<00:57, 24.55it/s]

...

[INFO] Skipping... Confidence below threshold: data/train\yunkyoung\RCPC_members-1302499576176238594-20200906_135112-img3.jpg
[INFO] Skipping... Confidence below threshold: data/train\yunkyoung\RCPC_members-1302499576176238594-20200906_135112-img4.jpg


 98%|████████████████████████████████████████████████████████████████████████████▍ | 1451/1480 [00:45<00:00, 32.95it/s]

[INFO] Skipping... Confidence below threshold: data/train\yunkyoung\RCPC_members-1307653534494519297-20200920_191112-img1.jpg
[INFO] Skipping... Confidence below threshold: data/train\yunkyoung\RCPC_members-1307653534494519297-20200920_191112-img2.jpg
[INFO] Skipping... Confidence below threshold: data/train\yunkyoung\RCPC_members-1307653534494519297-20200920_191112-img3.jpg
[INFO] Skipping... Confidence below threshold: data/train\yunkyoung\RCPC_members-1307653534494519297-20200920_191112-img4.jpg


100%|██████████████████████████████████████████████████████████████████████████████| 1480/1480 [00:46<00:00, 31.67it/s]

[STATUS] Face extraction process completed!


In [6]:
print('Initiating face extraction process (Validation)...')

SAVE = './data/faces/valid/'
CONFIDENCE = 0.5
valid_skipped = 0

count = 0
last = 'dahyun'
# training image
print('[STATUS] Reading & extracting image...')
for img in tqdm(test_img_glob):
    
    label = re.findall('\\\\[a-z].*\\\\', img)[0]
    label = label[1:-1]
    
    if not os.path.isdir(os.path.join(SAVE, label)):
        os.mkdir(os.path.join(SAVE, label))
    
    if label != last:
        count = 0
        last = label
    else:
        count = count + 1
    
    # read in the image
    a = cv2.imread(img)
    (h,w) = a.shape[:2]
    # create a blob object
    blob = cv2.dnn.blobFromImage(a, scalefactor=1.0, size=(224, 224), 
                                 mean=(104.0, 177.0, 123.0), swapRB=False, crop=False)
    face_model.setInput(blob)
    # detect face
    detector = face_model.forward() 
    
    # if any face is detected
    if len(detector) > 0:
        # get the index of the face
        i = np.argmax(detector[0,0,:,2])
        confidence = detector[0,0,i,2]
        
        # if the confidence is higher than the threshold we set earlier
        if confidence > CONFIDENCE:
            # extract face from a rectangle
            rect = detector[0,0,i,3:7] * np.array([w,h,w,h])
            start_x, start_y, end_x, end_y = rect.astype('int')
            
            face = a[start_y:end_y, start_x:end_x]
            
            # skip it if there is no face
            if face.size == 0:
                print('[INFO] Skipping... No face detected:', img)
                valid_skipped += 1
                continue
            else:
                img_file_name = label + '_face_' + str(count) + '_valid.jpg'
#                 print(SAVE + label + '/' + img_file_name)
#                 print(os.path.join(SAVE, label, img_file_name))
                cv2.imwrite(SAVE + label + '/' + img_file_name, face)
        
        # skip otherwise
        else:
            print('[INFO] Skipping... Confidence below threshold:', img)
            valid_skipped += 1
            continue

print('[STATUS] Face extraction process completed!')

  2%|█▏                                                                                | 5/330 [00:00<00:06, 46.51it/s]

Initiating face extraction process (Validation)...
[STATUS] Reading & extracting image...
[INFO] Skipping... Confidence below threshold: data/test\dahyun\RCPC_members-1205115119753023488-20191212_202009-img1.jpg


  8%|██████▍                                                                          | 26/330 [00:00<00:10, 28.88it/s]

[INFO] Skipping... Confidence below threshold: data/test\dahyun\RCPC_members-1232620519469400064-20200226_175647-img4.jpg


 22%|██████████████████▏                                                              | 74/330 [00:02<00:07, 36.12it/s]

...

[INFO] Skipping... Confidence below threshold: data/test\yunkyoung\RCPC_members-1232996082944077826-20200227_184908-img1.jpg
[INFO] Skipping... Confidence below threshold: data/test\yunkyoung\RCPC_members-1232996082944077826-20200227_184908-img2.jpg


 98%|██████████████████████████████████████████████████████████████████████████████▎ | 323/330 [00:10<00:00, 24.59it/s]

[INFO] Skipping... Confidence below threshold: data/test\yunkyoung\RCPC_members-1246761904338423813-20200405_182936-img1.jpg
[INFO] Skipping... Confidence below threshold: data/test\yunkyoung\RCPC_members-1263341003185909760-20200521_122901-img1.jpg


100%|████████████████████████████████████████████████████████████████████████████████| 330/330 [00:10<00:00, 30.02it/s]

[STATUS] Face extraction process completed!


In [7]:
train_face_glob = glob('data/faces/train/*/*.jpg')
valid_face_glob = glob('data/faces/valid/*/*.jpg')
print('-----Training Images-----')
print('Skipped images:', train_skipped)
print('Extracted faces:', len(train_face_glob))
print('-----Validation Images-----')
print('Skipped images:', valid_skipped)
print('Extracted faces:', len(valid_face_glob))

-----Training Images-----
Skipped images: 112
Extracted faces: 1368
-----Validation Images-----
Skipped images: 34
Extracted faces: 296
